# Intro
Inference notebook for [Hotel-ID starter - classification - traning](https://www.kaggle.com/code/michaln/hotel-id-starter-classification-traning)



# Setup

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

# Imports

In [ ]:
import numpy as np 
import pandas as pd 
import random
import os
import math

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm

import scipy

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam

# Global

In [ ]:
SEED = 42
IMG_SIZE = 512

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = "../input/hotel50k-id-512-data/images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

IMG_SIZE = 512

base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])


test_tta_transforms = {
    "base": A.Compose([A.ToFloat(),  APT.transforms.ToTensorV2(),]),
}

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pad_image(img)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE))


class HotelImageDataset:
    def __init__(self, data, transform=None, data_folder="train_images/"):
        self.data = data
        self.data_folder = data_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + record["image_id"]
        
        if "test" in self.data_folder:
            image = np.array(open_and_preprocess_image(image_path)).astype(np.uint8)
        else:
            image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
        
        return {
            "image" : transformed["image"],
        }

# Model

In [ ]:
# source: https://github.com/ronghuaiyang/arcface-pytorch/blob/master/models/metrics.py
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s

        return output

class HotelIdModel(nn.Module):
    def __init__(self, out_features, embed_size=256, backbone_name="densenet161"):
        super(HotelIdModel, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=False)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'head.flatten':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)

        self.arc_face = ArcMarginProduct(self.embed_size, out_features, s=30.0, m=0.50, easy_margin=False)

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
            nn.BatchNorm1d(self.embed_size),
        )

        print(f"Model {backbone_name} ArcMarginProduct - Features: {in_features}, Embeds: {self.embed_size}")
        
    def forward(self, input, targets = None):
        x = self.backbone(input)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return logits
        
        return x

# Model helper functions

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_embeds(loader, model, bar_desc="Generating embeds"):
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            output = model(input)
            outputs_all.extend(output.detach().cpu().numpy())
                        
    return outputs_all

In [ ]:
def get_distances(input, base_embeds, model_array):
    distances = None
    for i, model in enumerate(model_array):
        output = model(input)
        output = output.detach().cpu().numpy()
        model_base_embeds = base_embeds[i]
        output_distances = cosine_similarity(output, model_base_embeds)
        
        if distances is None:
            distances = output_distances
        else:
            distances = distances * output_distances
            
    return distances
    

def predict(loader, base_df, base_embeds, model_array, n_matches=5, bar_desc="Generating embeds"):
    preds = []
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            distances = get_distances(input, base_embeds, model_array)
            
            for j in range(len(distances)):
                tmp_df = base_df.copy()
                tmp_df["distance"] = distances[j]
                tmp_df = tmp_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
                preds.extend([tmp_df["hotel_id"].unique()[:n_matches]])

    return preds

def find_closest_match(args, test_loader, base_loader, model_array, n_matches=5):
    base_embeds = {}
    for i, model in enumerate(model_array):
        base_embeds[i] = get_embeds(base_loader, model, "Generating embeds for train")
    
    preds = predict(test_loader, base_loader.dataset.data, base_embeds, model_array, n_matches, f"Generating predictions")
        
    return preds

# Prepare data

In [ ]:
data_df = pd.read_csv("../input/train50kid-512-csv/train50k-id-512.csv")
sample_submission_df = pd.read_csv(PROJECT_FOLDER + "sample_submission.csv")
test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")

In [ ]:
# code hotel_id mapping created in training notebook by encoding hotel_ids
hotel_id_code_df = pd.read_csv('../input/hotelcodemapping-org/hotel_id_code_mapping.csv')
hotel_id_code_map = hotel_id_code_df.set_index('hotel_id_code').to_dict()["hotel_id"]

# Prepare model

In [ ]:
def get_model(model_type, backbone_name, embed_size, checkpoint_path, args):
    model = HotelIdModel(3116, embed_size, backbone_name)
        
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)
    
    return model

In [ ]:
class args:
    batch_size = 32
    num_workers = 2
    n_classes = data_df["hotel_id"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    
    
seed_everything(seed=SEED)

base_dataset = HotelImageDataset(data_df, base_transform, data_folder=TRAIN_DATA_FOLDER)
base_loader = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

test_dataset = HotelImageDataset(test_df, test_tta_transforms["base"], data_folder=TEST_DATA_FOLDER)
test_loader = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

In [ ]:
#Model array was altered depending on whether we were dealing with an ensemble, or a single model. In the latter case, the checkpoint file contained the trained model.

model_array = [get_model("arcmargin", 
                         "eca_nfnet_l0", 4096,
                         "../input/cp-eca-nfnet-l0-512-ep6-50k-id/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6.pt", 
                         args),
                          
               get_model("arcmargin", 
                         "tf_efficientnet_b0_ns", 4096,
                         "../input/cp-tf-eff-b0-ns-50k-id-ep9-512/checkpoint-arcmargin-model-tf_efficientnet_b0_ns-512x512-4096embeds-3116hotels-9.pt", 
                         args),
                             
               get_model("arcmargin", 
                         "eca_nfnet_l0", 4096,
                         "../input/cp-eca-512-ep6-rs-1/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6_rs1.pt", 
                         args),
                             
               get_model("arcmargin", 
                         "eca_nfnet_l0", 4096,
                         "../input/cp-eca-512-ep6-rs-23/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6_rs_23.pt", 
                         args),
                             
               get_model("arcmargin", 
                         "eca_nfnet_l0", 4096,
                         "../input/cp-eca-512-ep6-rs-2408/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6_rs_2408.pt", 
                         args),
               
               #get_model("arcmargin", 
                         #"eca_nfnet_l0", 4096,
                         #"../input/cp-eca-512-ep6-rs-242/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6_rs242.pt", 
                         #args),
               
               #get_model("arcmargin", 
                         #"eca_nfnet_l0", 4096,
                         #"../input/cp-eca-512-ep6-rs-824/checkpoint-arcmargin-model-eca_nfnet_l0-512x512-4096embeds-3116hotels-6_rs824.pt", 
                         #args),
              ]

# Submission

In [ ]:
if len(test_df) > 3:
    preds = find_closest_match(args, test_loader, base_loader, model_array, n_matches = 5)
    test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]

test_df.to_csv("submission.csv", index=False)
test_df.head()